In [1]:
import pandas as pd
import numpy as np

Insurance data

In [2]:
# load data
data = pd.read_csv("sfpuf2021_1_fall.csv").rename(columns = {"ADM_H_MEDSTA": "medicare_reason",
                                                             "DEM_AGE": "age",
                                                             "DEM_SEX": "sex",
                                                             "DEM_RACE": "race",
                                                             "HLT_OCPSYCHO": "mental_illness",
                                                             "ADM_DUAL_FLAG_YR": "medicaid",
                                                             "ADM_MA_FLAG_YR": "medicare_advantage",
                                                             "ADM_PARTD": "part_d",
                                                             "INS_D_PRIVNUM": "priv_1",
                                                             "INS_D_PVESI": "priv_2",
                                                             "INS_D_PVSELF": "priv_3",
                                                             "ACC_PAYPROB": "problem_paying_bills"})
data = data[["medicare_reason", "age", "sex", "race", "mental_illness", "medicaid", "medicare_advantage", "part_d", "priv_1", "priv_2", "priv_3", "problem_paying_bills"]]
data.head()

/var/folders/b3/rt6xbmt175g5bcvlg0sd0fqh0000gn/T/ipykernel_20777/3144253805.py:2: DtypeWarning: Columns (43,46,49,50,51,52,53,73,76,77,81,82,83,85,90,92,99,100,105,106,107,112,113,118,119,123,124,125,126,128,129,135,136,137,138,139,140,142,143,151,155,156,161,163,172,176,183,184,185,186,191,192,193,194,195,197,198,199,200,208,209,210,211,214,217,218,219,220,221,222,223,224,225,227,229) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("sfpuf2021_1_fall.csv").rename(columns = {"ADM_H_MEDSTA": "medicare_reason",


,medicare_reason,age,sex,race,mental_illness,medicaid,medicare_advantage,part_d,priv_1,priv_2,priv_3,problem_paying_bills
0,1,3,2,1,2,1,1,2,1,2,1,2
1,1,2,1,2,2,1,1,1,1,2,1,2
2,1,3,2,1,2,1,1,1,1,2,1,2
3,1,3,2,1,2,1,1,1,1,2,1,2
4,1,2,2,3,2,3,3,1,2,2,2,1


In [ ]:
# clean relevant columns
data["medicare_reason"] = data["medicare_reason"].map({1: "Aged", 2: "Disabled", 3: pd.NA})
data["age"] = data["age"].map({1: "<65", 2: "65-74", 3: "75+"})
data["sex"] = data["sex"].map({1: "Male", 2: "Female"})
data["race"] = data["race"].map({1: "White", 2: "Black", 3: "Hispanic", 4: "Other"})
data["mental_illness"] = data["mental_illness"].map({1: "Yes", "1": "Yes", 2: "No", "2": "No", "D": pd.NA, "R": pd.NA})
data["medicaid"] = data["medicaid"].map({1: 0, 2: 1, 3: 1})
data["medicare_advantage"] = data["medicare_advantage"].map({1: 0, 2: 1, 3: 1})
data["part_d"] = data["part_d"].map({1: 1, 2: 0})
data["priv_1"] = data["priv_1"].map({1: 1, 2: 0})
data["priv_2"] = data["priv_2"].map({1: 1, 2: 0})
data["priv_3"] = data["priv_3"].map({1: 1, 2: 0})
data["private_insurance"] = data[["priv_1", "priv_2", "priv_3"]].sum(axis = 1)
data["private_insurance"] = np.where(data["private_insurance"] > 1, 1, 0)
data = data.drop(columns = ["priv_1", "priv_2", "priv_3"])
data["problem_paying_bills"] = data["problem_paying_bills"].map({1: 1, "1": 1, 2: 0, "2": 0}).astype("Int64")
data.head()

,medicare_reason,age,sex,race,mental_illness,medicaid,medicare_advantage,part_d,problem_paying_bills,private_insurance
0,Aged,75+,Female,White,No,0,0,0,0,1
1,Aged,65-74,Male,Black,No,0,0,1,0,1
2,Aged,75+,Female,White,No,0,0,1,0,1
3,Aged,75+,Female,White,No,0,0,1,0,1
4,Aged,65-74,Female,Hispanic,No,1,1,1,1,0


In [4]:
# export to csv
data.to_csv("rachel_data_cleaned.csv")

Map data preprocessing

In [5]:
# load and filter data
map_data = pd.read_csv("Medicare Monthly Enrollment Data_July 2024.csv")
map_data_filtered = map_data.loc[((map_data["YEAR"] == 2021) & 
                                 (map_data["MONTH"] == "Year") & 
                                 (map_data["BENE_GEO_LVL"] == "State") &
                                 (~map_data["BENE_STATE_DESC"].isin(["American Samoa", "Guam", "Northern Mariana Islands", "District of Columbia",
                                                                     "Puerto Rico", "Virgin Islands", "Unknown", "Foreign and Other Outlying Areas"])))]

map_data_filtered.head()

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,B_TOT_BENES,B_ORGNL_MDCR_BENES,B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,PRSCRPTN_DRUG_DEEMED_ELIGIBLE_FULL_LIS_BENES,PRSCRPTN_DRUG_FULL_LIS_BENES,PRSCRPTN_DRUG_PARTIAL_LIS_BENES,PRSCRPTN_DRUG_NO_LIS_BENES
346867,2021,Year,State,AL,Alabama,Total,01,1070474.0,528983.0,541491.0,...,997828.0,456357.0,541470.0,806300.0,290817.0,515483.0,218199.0,32582.0,8713.0,546806.0
346936,2021,Year,State,AK,Alaska,Total,02,108116.0,105831.0,2286.0,...,95519.0,93234.0,2285.0,70012.0,68782.0,1230.0,20276.0,900.0,170.0,48666.0
346968,2021,Year,State,AZ,Arizona,Total,04,1400160.0,767513.0,632647.0,...,1295512.0,662898.0,632614.0,1068454.0,467625.0,600829.0,244828.0,12576.0,5974.0,805075.0
346985,2021,Year,State,AR,Arkansas,Total,05,653277.0,430724.0,222553.0,...,607318.0,384774.0,222544.0,482450.0,274386.0,208064.0,139370.0,16579.0,5501.0,321000.0
347062,2021,Year,State,CA,California,Total,06,6499203.0,3436742.0,3062462.0,...,5938760.0,2876785.0,3061975.0,5233277.0,2273313.0,2959965.0,1566827.0,80173.0,18080.0,3568197.0


In [6]:
# clean data
map_data_cleaned = map_data_filtered.rename(columns = {"YEAR": "year",
                                                        "BENE_STATE_DESC": "state",
                                                        "TOT_BENES": "total",
                                                        "AGED_TOT_BENES": "aged",
                                                        "DSBLD_TOT_BENES": "disabled",
                                                        "MALE_TOT_BENES": "male",
                                                        "FEMALE_TOT_BENES": "female",
                                                        "WHITE_TOT_BENES": "white",
                                                        "BLACK_TOT_BENES": "black",
                                                        "HSPNC_TOT_BENES": "hispanic",
                                                        "API_TOT_BENES": "other_API",
                                                        "NATIND_TOT_BENES": "other_Ind",
                                                        "OTHR_TOT_BENES": "other_other", 
                                                        "AGE_LT_25_BENES": "l65_l25",
                                                        "AGE_25_TO_44_BENES":  "l65_25_44",
                                                        "AGE_45_TO_64_BENES": "l65_45_64",
                                                        "AGE_65_TO_69_BENES": "65_74_65_69",
                                                        "AGE_70_TO_74_BENES": "65_74_70_74",
                                                        "AGE_75_TO_79_BENES": "75p_75_79", 
                                                        "AGE_80_TO_84_BENES": "75p_80_84",
                                                        "AGE_85_TO_89_BENES": "75p_85_89",
                                                        "AGE_90_TO_94_BENES": "75p_90_94",
                                                        "AGE_GT_94_BENES": "75p_95p"})
map_data_cleaned["other"] = map_data_cleaned[["other_API", "other_Ind", "other_other"]].apply(pd.to_numeric, errors='coerce', axis=1).sum(axis = 1)
map_data_cleaned["less_than_65"] = map_data_cleaned[["l65_l25", "l65_25_44", "l65_45_64"]].apply(pd.to_numeric, errors='coerce', axis=1).sum(axis = 1)
map_data_cleaned["between_65_to_74"] = map_data_cleaned[["65_74_65_69", "65_74_70_74"]].apply(pd.to_numeric, errors='coerce', axis=1).sum(axis = 1)
map_data_cleaned["greater_than_74"] = map_data_cleaned[["75p_75_79", "75p_80_84", "75p_85_89", "75p_90_94", "75p_95p"]].apply(pd.to_numeric, errors='coerce', axis=1).sum(axis = 1)
map_data_cleaned = map_data_cleaned[["year", "state", "total", "aged", "disabled", "male", "female", "white", "black", "hispanic", "other", "less_than_65", "between_65_to_74", "greater_than_74"]]
map_data_cleaned.head()

,year,state,total,aged,disabled,male,female,white,black,hispanic,other,less_than_65,between_65_to_74,greater_than_74
346867,2021,Alabama,1070474.0,866594.0,203880.0,480691.0,589783.0,787616.0,253074.0,9340.0,20445.0,203880.0,506608.0,359987.0
346936,2021,Alaska,108116.0,96353.0,11764.0,54672.0,53445.0,79800.0,2834.0,3111.0,22371.0,11764.0,62607.0,33745.0
346968,2021,Arizona,1400160.0,1256668.0,143492.0,650142.0,750018.0,1074842.0,46430.0,184964.0,93924.0,143492.0,712471.0,544197.0
346985,2021,Arkansas,653277.0,527679.0,125599.0,299930.0,353347.0,542021.0,84598.0,11843.0,14815.0,125599.0,302013.0,225667.0
347062,2021,California,6499203.0,5896417.0,602787.0,2980289.0,3518915.0,3548461.0,405592.0,1441908.0,1103243.0,602786.0,3316864.0,2579554.0


In [7]:
# convert relevant columns to percents
def convert_to_percent (column_name):
    return round((pd.to_numeric(map_data_cleaned[column_name])/pd.to_numeric(map_data_cleaned["total"])) * 100, ndigits = 1)

columns = ["aged", "disabled", "male", "female", "white", "black", "hispanic", "other", "less_than_65", "between_65_to_74", "greater_than_74"]
for column in columns:
    map_data_cleaned[column + "_pct"] = convert_to_percent(column)

map_data_cleaned.head()

,year,state,total,aged,disabled,male,female,white,black,hispanic,...,disabled_pct,male_pct,female_pct,white_pct,black_pct,hispanic_pct,other_pct,less_than_65_pct,between_65_to_74_pct,greater_than_74_pct
346867,2021,Alabama,1070474.0,866594.0,203880.0,480691.0,589783.0,787616.0,253074.0,9340.0,...,19.0,44.9,55.1,73.6,23.6,0.9,1.9,19.0,47.3,33.6
346936,2021,Alaska,108116.0,96353.0,11764.0,54672.0,53445.0,79800.0,2834.0,3111.0,...,10.9,50.6,49.4,73.8,2.6,2.9,20.7,10.9,57.9,31.2
346968,2021,Arizona,1400160.0,1256668.0,143492.0,650142.0,750018.0,1074842.0,46430.0,184964.0,...,10.2,46.4,53.6,76.8,3.3,13.2,6.7,10.2,50.9,38.9
346985,2021,Arkansas,653277.0,527679.0,125599.0,299930.0,353347.0,542021.0,84598.0,11843.0,...,19.2,45.9,54.1,83.0,12.9,1.8,2.3,19.2,46.2,34.5
347062,2021,California,6499203.0,5896417.0,602787.0,2980289.0,3518915.0,3548461.0,405592.0,1441908.0,...,9.3,45.9,54.1,54.6,6.2,22.2,17.0,9.3,51.0,39.7


In [8]:
# export to csv
map_data_cleaned.to_csv("rachel_map_data_cleaned.csv")